In [2]:
import numpy as np
import pandas as pd

In [3]:
pd_excel = pd.ExcelFile("Edulyt India - Data for Project - 4.xlsx")

In [4]:
pd_excel.sheet_names

['New_Account',
 'New_Card',
 'District',
 'Loan',
 'New_Client',
 'New_Disposition',
 'New_Transaction',
 'Order']

In [5]:
df_Account = pd_excel.parse('New_Account')
df_Card = pd_excel.parse('New_Card')
df_District = pd_excel.parse('District')
df_Loan = pd_excel.parse('Loan')
df_Client = pd_excel.parse('New_Client')
df_Disposition = pd_excel.parse('New_Disposition')
df_Transaction = pd_excel.parse('New_Transaction')
df_Order = pd_excel.parse('Order')


In [6]:
df_Account.isnull().sum()

account_id     0
district_id    0
frequency      0
dtype: int64

In [7]:
df_Card.isnull().sum()

card_id    0
disp_id    0
type       0
issued     0
dtype: int64

In [8]:
df_District.isnull().sum()

A1     0
A2     0
A3     0
A4     0
A5     0
A6     0
A7     0
A8     0
A9     0
A10    0
A11    0
A12    0
A13    0
A14    0
A15    0
A16    0
dtype: int64

In [9]:
df_Loan.isnull().sum()

loan_id       0
account_id    0
date          0
amount        0
duration      0
payments      0
status        0
dtype: int64

In [10]:
df_Client.isnull().sum()

Unnamed: 0      0
client_id       0
birth_number    0
district_id     0
gender          0
age             0
age_levels      0
dtype: int64

In [11]:
df_Disposition.isnull().sum()

disp_id       0
client_id     0
account_id    0
type          0
dtype: int64

In [12]:
df_Transaction.isnull().sum()

Unnamed: 0         0
account_id         0
date               0
type               0
operation     178663
amount             0
balance            0
k_symbol      478646
dtype: int64

In [13]:
df_Order["k_symbol"] = df_Order["k_symbol"].fillna(method='ffill')

C:\Users\91979\AppData\Local\Temp\ipykernel_328\1299858480.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_Order["k_symbol"] = df_Order["k_symbol"].fillna(method='ffill')


In [14]:
df_Order.isnull().sum()

order_id      0
account_id    0
bank_to       0
account_to    0
amount        0
k_symbol      0
dtype: int64

In [15]:
df_Transaction.operation.value_counts(), df_Transaction.isnull().sum()

(operation
 CASH WITHDRAWAL                 432377
 REMITTANCE TO ANOTHER BANK      208283
 CREDIT IN CASH                  156320
 COLLECTION FROM ANOTHER BANK     65226
 CREDIT CARD WITHDRAWAL            7706
 Name: count, dtype: int64,
 Unnamed: 0         0
 account_id         0
 date               0
 type               0
 operation     178663
 amount             0
 balance            0
 k_symbol      478646
 dtype: int64)

In [16]:
df_Transaction["operation"] = df_Transaction.operation.fillna("CREDIT IN CASH ")

In [17]:
df_Transaction.operation.value_counts(), df_Transaction.isnull().sum()

(operation
 CASH WITHDRAWAL                 432377
 REMITTANCE TO ANOTHER BANK      208283
 CREDIT IN CASH                  178663
 CREDIT IN CASH                  156320
 COLLECTION FROM ANOTHER BANK     65226
 CREDIT CARD WITHDRAWAL            7706
 Name: count, dtype: int64,
 Unnamed: 0         0
 account_id         0
 date               0
 type               0
 operation          0
 amount             0
 balance            0
 k_symbol      478646
 dtype: int64)

In [18]:
len(df_Transaction), len(df_Order)

(1048575, 6471)

In [19]:
df_Transaction = df_Transaction.merge(
    df_Order[['account_id', 'k_symbol']],
    on='account_id',
    how='left',
    suffixes=('', '_order')
)

# Fill missing k_symbol in transaction using k_symbol from order
df_Transaction['k_symbol'] = df_Transaction['k_symbol'].fillna(df_Transaction['k_symbol_order'])

# Drop the extra merged column
df_Transaction.drop(columns=['k_symbol_order'], inplace=True)

In [20]:
df_Transaction = df_Transaction.drop("Unnamed: 0",axis = 1)

In [21]:
df_Transaction.head()

,account_id,date,type,operation,amount,balance,k_symbol
0,2378,930101,CREDIT,CREDIT IN CASH,700.0,700.0,SIPO
1,576,930101,CREDIT,CREDIT IN CASH,900.0,900.0,SIPO
2,704,930101,CREDIT,CREDIT IN CASH,1000.0,1000.0,SIPO
3,704,930101,CREDIT,CREDIT IN CASH,1000.0,1000.0,SIPO
4,3818,930101,CREDIT,CREDIT IN CASH,600.0,600.0,SIPO


In [22]:
df_District.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,1,Hl.m. Praha,Prague,1204953,0,0,0,1,1,100.0,12541,0.29,0.43,167,85677,99107
1,2,Benesov,central Bohemia,88884,80,26,6,2,5,46.7,8507,1.67,1.85,132,2159,2674
2,3,Beroun,central Bohemia,75232,55,26,4,1,5,41.7,8980,1.95,2.21,111,2824,2813
3,4,Kladno,central Bohemia,149893,63,29,6,2,6,67.4,9753,4.64,5.05,109,5244,5892
4,5,Kolin,central Bohemia,95616,65,30,4,1,6,51.4,9307,3.85,4.43,118,2616,3040


In [23]:
df_District = df_District.rename(columns = {'A1':'Code',
                                            'A2':'Name',
                                            'A3':'Region',
                                            'A4':'Population',
                                            'A5':'Municipalities<499',
                                            'A6':'Municipalities BW 500-1999',
                                           'A7':'Municipalities BW 2000-9999',
                                            'A8':'Municipalities>10000',
                                            'A9': 'Cities',
                                            'A10':'Urban Ration',
                                            'A11':'AVG_SALARY',
                                           'A12':'Unemployment 95',
                                            'A13':'Unemployment 96',
                                            'A14':'entrepreneurs per 1000',
                                            'A15': 'Crimes 95',
                                            'A16': 'Crimes 96'})

In [24]:
df_District.head()

,Code,Name,Region,Population,Municipalities<499,Municipalities BW 500-1999,Municipalities BW 2000-9999,Municipalities>10000,Cities,Urban Ration,AVG_SALARY,Unemployment 95,Unemployment 96,entrepreneurs per 1000,Crimes 95,Crimes 96
0,1,Hl.m. Praha,Prague,1204953,0,0,0,1,1,100.0,12541,0.29,0.43,167,85677,99107
1,2,Benesov,central Bohemia,88884,80,26,6,2,5,46.7,8507,1.67,1.85,132,2159,2674
2,3,Beroun,central Bohemia,75232,55,26,4,1,5,41.7,8980,1.95,2.21,111,2824,2813
3,4,Kladno,central Bohemia,149893,63,29,6,2,6,67.4,9753,4.64,5.05,109,5244,5892
4,5,Kolin,central Bohemia,95616,65,30,4,1,6,51.4,9307,3.85,4.43,118,2616,3040


In [25]:
df_District.isnull().sum()

Code                           0
Name                           0
Region                         0
Population                     0
Municipalities<499             0
Municipalities BW 500-1999     0
Municipalities BW 2000-9999    0
Municipalities>10000           0
Cities                         0
Urban Ration                   0
AVG_SALARY                     0
Unemployment 95                0
Unemployment 96                0
entrepreneurs per 1000         0
Crimes 95                      0
Crimes 96                      0
dtype: int64

## Analysis on Highly populated versus Low populated districts. Find out the amount of Credit and Debit transaction from 5 highly populated and 5 lowest populated areas respectively, above analysis should be from last 3 months

In [27]:
df_District_Subset = df_District[['Code','Name','Population']].copy()
top_5_populated = df_District_Subset.sort_values(by = 'Population', ascending = False).head(5)
bottom_5_populated = df_District_Subset.sort_values(by = 'Population', ascending = True).head(5)
target_districts = pd.concat([top_5_populated, bottom_5_populated])
df_Transaction['date'] = pd.to_datetime(df_Transaction['date'], format='%y%m%d', errors='coerce')
max_date = df_Transaction['date'].max()
cutoff_date = max_date - pd.DateOffset(months=3)
df_recent = df_Transaction[df_Transaction['date'] >= cutoff_date]
df_account_subset = df_Account[['account_id', 'district_id']]
df_merged = df_recent.merge(df_account_subset, on='account_id', how='left')
df_merged = df_merged.merge(df_District[['Code', 'Name', 'Population']], left_on='district_id', right_on='Code', how='left')
df_filtered = df_merged[df_merged['district_id'].isin(target_districts['Code'])].copy()
result = df_filtered.groupby(['Name', 'type'])['amount'].sum().unstack(fill_value=0).reset_index()
result = result.rename(columns={'Name': 'District Name', 'CREDIT': 'Total Credit', 'DEBIT': 'Total Debit'})
result['Population Type'] = result['District Name'].apply(
    lambda x: 'High Population' if x in top_5_populated['Name'].values else 'Low Population'
)
result = result[['Population Type', 'District Name', 'Total Credit', 'Total Debit']]
result = result.sort_values(['Population Type', 'Total Credit'], ascending=[True, False]).reset_index(drop=True)
print(result)

type  Population Type    District Name  Total Credit  Total Debit
0     High Population      Hl.m. Praha    57722834.7   53289950.6
1     High Population  Ostrava - mesto    14004861.9   12942882.7
2     High Population          Karvina    13726873.4   12390863.0
3     High Population  Frydek - Mistek    11252697.2   10250525.9
4     High Population     Brno - mesto     9886866.7    9341458.7
5      Low Population       Prachatice     6826829.0    6410053.1
6      Low Population          Jesenik     4022964.0    3706207.8
7      Low Population           Tachov     4006482.4    3985514.6
8      Low Population         Rakovnik     3705149.9    3233925.2
9      Low Population         Rokycany     3280197.3    3079345.7


In [28]:
df_Card.head()

,card_id,disp_id,type,issued
0,1005,9285,CLASSIC,931107
1,104,588,CLASSIC,940119
2,747,4915,CLASSIC,940205
3,70,439,CLASSIC,940208
4,577,3687,CLASSIC,940215


In [29]:
df_Client.head()

,Unnamed: 0,client_id,birth_number,district_id,gender,age,age_levels
0,1,1,701213,18,FEMALE,29,ADULT
1,2,2,450204,1,MALE,54,MIDDLE AGED
2,3,3,401009,1,FEMALE,59,MIDDLE AGED
3,4,4,561201,5,MALE,43,MIDDLE AGED
4,5,5,600703,5,FEMALE,39,MIDDLE AGED


## How many cards are issued to middle aged females

In [31]:
mid_age_females = df_Client[(df_Client['gender'] == 'FEMALE') & (df_Client['age_levels'] == 'MIDDLE AGED')]
female_disp = df_Disposition[df_Disposition['client_id'].isin(mid_age_females['client_id'])]
cards_issued = df_Card[df_Card['disp_id'].isin(female_disp['disp_id'])]
print("Cards issued to middle-aged females:", len(cards_issued))

Cards issued to middle-aged females: 227


In [32]:
df_District.head() 

,Code,Name,Region,Population,Municipalities<499,Municipalities BW 500-1999,Municipalities BW 2000-9999,Municipalities>10000,Cities,Urban Ration,AVG_SALARY,Unemployment 95,Unemployment 96,entrepreneurs per 1000,Crimes 95,Crimes 96
0,1,Hl.m. Praha,Prague,1204953,0,0,0,1,1,100.0,12541,0.29,0.43,167,85677,99107
1,2,Benesov,central Bohemia,88884,80,26,6,2,5,46.7,8507,1.67,1.85,132,2159,2674
2,3,Beroun,central Bohemia,75232,55,26,4,1,5,41.7,8980,1.95,2.21,111,2824,2813
3,4,Kladno,central Bohemia,149893,63,29,6,2,6,67.4,9753,4.64,5.05,109,5244,5892
4,5,Kolin,central Bohemia,95616,65,30,4,1,6,51.4,9307,3.85,4.43,118,2616,3040


In [33]:
df_Account.head()

,account_id,district_id,frequency
0,576,55,MONTHLY ISSUANCE
1,3818,74,MONTHLY ISSUANCE
2,704,55,MONTHLY ISSUANCE
3,2378,16,MONTHLY ISSUANCE
4,2632,24,MONTHLY ISSUANCE


In [34]:
df_Disposition.head()

,disp_id,client_id,account_id,type
0,1,1,1,OWNER
1,2,2,2,OWNER
2,3,3,2,USER
3,4,4,3,OWNER
4,5,5,3,USER


In [35]:
df_Card.head()

,card_id,disp_id,type,issued
0,1005,9285,CLASSIC,931107
1,104,588,CLASSIC,940119
2,747,4915,CLASSIC,940205
3,70,439,CLASSIC,940208
4,577,3687,CLASSIC,940215


## Number of cards issued in district where average salary is more than 9000, is it a good strategy?

In [37]:
high_salary_districts = df_District[df_District['AVG_SALARY'] > 9000]['Code']
high_salary_accounts = df_Account[df_Account['district_id'].isin(high_salary_districts)]['account_id']
high_salary_disps = df_Disposition[df_Disposition['account_id'].isin(high_salary_accounts)]['disp_id']
cards_issued = df_Card[df_Card['disp_id'].isin(high_salary_disps)]
print("Number of cards issued in high average salary districts:", len(cards_issued))

Number of cards issued in high average salary districts: 437


## Are we providing loans to members belonging to district where committed crimes are more than 6000 for code 95, if yes then provide the number of loans per district?

In [39]:
# Convert crimes column to numeric
df_District['Crimes 95'] = pd.to_numeric(df_District['Crimes 95'], errors='coerce')

# Step 1: Filter high-crime districts
high_crime_districts = df_District[df_District['Crimes 95'] > 6000][['Code', 'Name']]

# Step 2: Accounts in those districts
accounts_in_high_crime = df_Account[df_Account['district_id'].isin(high_crime_districts['Code'])]

# Step 3: Loans linked to those accounts
loans_in_high_crime = df_Loan[df_Loan['account_id'].isin(accounts_in_high_crime['account_id'])]

# Step 4: Merge account info
accounts_with_district = accounts_in_high_crime[['account_id', 'district_id']]
loan_district_info = loans_in_high_crime.merge(accounts_with_district, on='account_id', how='left')
loan_district_info = loan_district_info.merge(high_crime_districts, left_on='district_id', right_on='Code', how='left')

# Step 5: Count loans per district
loan_count_per_district = loan_district_info.groupby('Name')['loan_id'].count().reset_index()
loan_count_per_district.columns = ['District Name', 'Number of Loans']

print(loan_count_per_district)


      District Name  Number of Loans
0      Brno - mesto               24
1  Ceske Budejovice                8
2       Hl.m. Praha               84
3           Karvina               24
4           Olomouc               14
5   Ostrava - mesto               20
6         Pardubice               10
7     Plzen - mesto                6
8           Teplice                6
9    Usti nad Labem                3


## How much money was collected from other banks for customer belongs to districts where unemployment rate for any year is greater than 2%.

In [41]:
df_District['Unemployment 95'] = pd.to_numeric(df_District['Unemployment 95'], errors='coerce')
df_District['Unemployment 96'] = pd.to_numeric(df_District['Unemployment 96'], errors='coerce')
high_unemp_districts = df_District[
    (df_District['Unemployment 95'] > 2) | (df_District['Unemployment 96'] > 2)
]['Code']
accounts_high_unemp = df_Account[df_Account['district_id'].isin(high_unemp_districts)]['account_id']
df_Transaction['amount'] = pd.to_numeric(df_Transaction['amount'], errors='coerce')
inflow_trans = df_Transaction[
    (df_Transaction['account_id'].isin(accounts_high_unemp)) &
    (df_Transaction['type'] == 'Credit') &
    (df_Transaction['k_symbol'].notnull())
]
total_collected = inflow_trans['amount'].sum()
print("Total amount collected from other banks (unemployment > 2%):", total_collected)


Total amount collected from other banks (unemployment > 2%): 0.0


## Create profile of customers in accordance of districts where max money is being paid to
* a. Insurance.
* b. Household
* c. Leasing
* d. Loan

In [43]:
df_Transaction['amount'] = pd.to_numeric(df_Transaction['amount'], errors='coerce')
df_acc_dist = df_Account[['account_id', 'district_id']]
df_trans_dist = df_Transaction.merge(df_acc_dist, on='account_id', how='left')
df_trans_dist = df_trans_dist.merge(df_District[['Code', 'Name']], left_on='district_id', right_on='Code', how='left')

# Clean and standardize k_symbol just in case
df_trans_dist['k_symbol'] = df_trans_dist['k_symbol'].str.strip().str.upper()

category_map = {
    'Insurance': 'POJISTNE',
    'Household': 'SLUZBY',
    'Leasing': 'LEASING',
    'Loan': 'UVER'
}

customer_profiles = {}

for category, symbol in category_map.items():
    df_cat = df_trans_dist[df_trans_dist['k_symbol'] == symbol]

    if df_cat.empty:
        print(f"⚠️ No transactions found for category: {category}")
        continue

    district_sum = df_cat.groupby(['district_id', 'Name'])['amount'].sum().reset_index()

    if district_sum.empty:
        print(f"⚠️ No district sums found for category: {category}")
        continue

    max_district_row = district_sum.sort_values('amount', ascending=False).head(1)
    max_district_id = max_district_row['district_id'].values[0]

    accounts_in_district = df_Account[df_Account['district_id'] == max_district_id]['account_id']
    disp_in_district = df_Disposition[df_Disposition['account_id'].isin(accounts_in_district)]
    clients = df_Client[df_Client['client_id'].isin(disp_in_district['client_id'])]

    if clients.empty:
        print(f"⚠️ No clients found for max district in category: {category}")
        continue

    profile_summary = clients.groupby(['gender', 'age_levels']).size().reset_index(name='count')

    customer_profiles[category] = {
        'district': max_district_row['Name'].values[0],
        'total_amount': max_district_row['amount'].values[0],
        'profile': profile_summary
    }

# Final output
for category, data in customer_profiles.items():
    print(f"\n=== {category} ===")
    print(f"District: {data['district']}")
    print(f"Total Amount Paid: ₹{data['total_amount']:.2f}")
    print("Customer Profile:")
    print(data['profile'])


⚠️ No transactions found for category: Household

=== Insurance ===
District: Hl.m. Praha
Total Amount Paid: ₹136619951.00
Customer Profile:
   gender   age_levels  count
0  FEMALE        ADULT     55
1  FEMALE  MIDDLE AGED    173
2  FEMALE       SENIOR     53
3  FEMALE        YOUTH     53
4    MALE        ADULT     66
5    MALE  MIDDLE AGED    173
6    MALE       SENIOR     50
7    MALE        YOUTH     48

=== Leasing ===
District: Hl.m. Praha
Total Amount Paid: ₹97130951.00
Customer Profile:
   gender   age_levels  count
0  FEMALE        ADULT     55
1  FEMALE  MIDDLE AGED    173
2  FEMALE       SENIOR     53
3  FEMALE        YOUTH     53
4    MALE        ADULT     66
5    MALE  MIDDLE AGED    173
6    MALE       SENIOR     50
7    MALE        YOUTH     48

=== Loan ===
District: Hl.m. Praha
Total Amount Paid: ₹180978779.00
Customer Profile:
   gender   age_levels  count
0  FEMALE        ADULT     55
1  FEMALE  MIDDLE AGED    173
2  FEMALE       SENIOR     53
3  FEMALE        YOUTH 

## Create profile of customers in accordance of districts for the status of loan payment, there will be 4 categories

In [45]:
# Map loan status
loan_status_map = {
    0: 'Defaulted',
    1: 'Paid with Delay',
    2: 'Paid on Time'
}

# Create status column
df_Loan['loan_status'] = df_Loan['status'].map(loan_status_map)
df_Loan['loan_status'] = df_Loan['loan_status'].fillna('Ongoing')

# Merge loan with account → district info
df_loan_acc = df_Loan.merge(df_Account[['account_id', 'district_id']], on='account_id', how='left')
df_loan_acc = df_loan_acc.merge(df_District[['Code', 'Name']], left_on='district_id', right_on='Code', how='left')

# Merge with disposition to get client_id
df_disp = df_Disposition[['account_id', 'client_id']]
df_loan_disp = df_loan_acc.merge(df_disp, on='account_id', how='left')

# Merge with client to get age and gender
df_loan_client = df_loan_disp.merge(df_Client[['client_id', 'gender', 'age_levels']], on='client_id', how='left')

# Group by district and loan_status with gender and age
profile = df_loan_client.groupby(['Name', 'loan_status', 'gender', 'age_levels']).size().reset_index(name='count')

# Rename for clarity
profile = profile.rename(columns={
    'Name': 'District',
    'loan_status': 'Loan Status',
    'gender': 'Gender',
    'age_level': 'Age Group',
    'count': 'Number of Customers'
})

# Show result
profile.head()


,District,Loan Status,Gender,age_levels,Number of Customers
0,Benesov,Ongoing,FEMALE,MIDDLE AGED,4
1,Benesov,Ongoing,FEMALE,YOUTH,1
2,Benesov,Ongoing,MALE,ADULT,1
3,Benesov,Ongoing,MALE,MIDDLE AGED,1
4,Beroun,Ongoing,FEMALE,ADULT,3


## Relate the output of above with district conditions like Crime, Unemployment Rate and Average Salary.

In [47]:
df_district_summary = df_District[['Name', 'AVG_SALARY', 'Unemployment 95', 'Unemployment 96', 'Crimes 95', 'Crimes 96']]
df_district_summary = df_district_summary.rename(columns={
    'Name': 'District',
    'AVG_SALARY': 'Average Salary',
    'Unemployment 95': 'Unemployment 95 (%)',
    'Unemployment 96': 'Unemployment 96 (%)',
    'Crimes 95': 'Crimes 1995',
    'Crimes 96': 'Crimes 1996'
})

# Merge with the customer loan profile
profile_enriched = profile.merge(df_district_summary, on='District', how='left')

# Display final output
profile_enriched.head()

,District,Loan Status,Gender,age_levels,Number of Customers,Average Salary,Unemployment 95 (%),Unemployment 96 (%),Crimes 1995,Crimes 1996
0,Benesov,Ongoing,FEMALE,MIDDLE AGED,4,8507,1.67,1.85,2159.0,2674
1,Benesov,Ongoing,FEMALE,YOUTH,1,8507,1.67,1.85,2159.0,2674
2,Benesov,Ongoing,MALE,ADULT,1,8507,1.67,1.85,2159.0,2674
3,Benesov,Ongoing,MALE,MIDDLE AGED,1,8507,1.67,1.85,2159.0,2674
4,Beroun,Ongoing,FEMALE,ADULT,3,8980,1.95,2.21,2824.0,2813


## Owners from which district are issuing permanent orders and asking for a loan.

In [49]:
# Step 1: Get all owners
owners = df_Disposition[df_Disposition['type'].str.upper() == 'OWNER']

# Step 2: Filter orders made by owners
orders_by_owners = df_Order[df_Order['account_id'].isin(owners['account_id'])]

# Step 3: Accounts with permanent orders (i.e., orders with non-null `k_symbol`)
perm_order_accounts = orders_by_owners[orders_by_owners['k_symbol'].notnull()]['account_id'].unique()

# Step 4: Accounts with loans
loan_accounts = df_Loan['account_id'].unique()

# Step 5: Accounts with both permanent orders and loans
common_accounts = set(perm_order_accounts).intersection(set(loan_accounts))

# Step 6: Get corresponding districts
accounts_with_district = df_Account[df_Account['account_id'].isin(common_accounts)]

# Step 7: Merge with district names
accounts_with_district = accounts_with_district.merge(df_District[['Code', 'Name']], left_on='district_id', right_on='Code', how='left')

# Step 8: Group by district and count
result = accounts_with_district['Name'].value_counts().reset_index()
result.columns = ['District', 'Number of Owners with Permanent Orders & Loans']

# Show result
result.head()


,District,Number of Owners with Permanent Orders & Loans
0,Hl.m. Praha,84
1,Brno - mesto,24
2,Karvina,24
3,Ostrava - mesto,20
4,Zlin,17


## Can we say customers from Bohemia are the ones having more male customers possessing Gold cards in comparison of Moravia?

In [51]:
gold_cards = df_Card[df_Card['type'].str.upper() == 'GOLD']
owners = df_Disposition[df_Disposition['type'].str.upper() == 'OWNER']
gold_owners = gold_cards.merge(owners, on='disp_id', how='inner')
gold_owners = gold_owners.merge(df_Client[['client_id', 'gender']], on='client_id', how='left')
gold_owners = gold_owners.merge(df_Account[['account_id', 'district_id']], on='account_id', how='left')
gold_owners = gold_owners.merge(df_District[['Code', 'Region']], left_on='district_id', right_on='Code', how='left')
gold_males = gold_owners[gold_owners['gender'].str.upper() == 'MALE']
result = gold_males['Region'].value_counts().reset_index()
result.columns = ['Region', 'Number of Male Gold Cardholders']
result.head()

,Region,Number of Male Gold Cardholders
0,north Moravia,12
1,south Moravia,10
2,north Bohemia,7
3,Prague,6
4,west Bohemia,5


## How many customers having credit card are also availing the loan facilities.

In [53]:
credit_cards = df_Card[df_Card['type'].str.upper() == 'CREDIT']
credit_card_disp = credit_cards.merge(df_Disposition, left_on='disp_id', right_on='disp_id', how='inner')
credit_card_clients = credit_card_disp[['client_id']].drop_duplicates()
loan_accounts = df_Loan['account_id'].unique()
loan_clients = df_Disposition[df_Disposition['account_id'].isin(loan_accounts)][['client_id']]
common_clients = pd.merge(credit_card_clients, loan_clients, on='client_id',how = 'inner').drop_duplicates()
common_clients

,client_id


## Can we say that customers having Classic and Junior card are the ones who are more in debt.

In [55]:
df_Card['type'] = df_Card['type'].str.upper().str.strip()
card_with_disp = df_Card.merge(df_Disposition, on='disp_id', how='left')
card_loans = card_with_disp.merge(df_Loan, on='account_id', how='inner')
card_debt = card_loans.groupby('type_x')['amount'].sum().reset_index().sort_values(by='amount', ascending=False)
card_debt.columns = ['Card Type', 'Total Debt (Loan Amount)']

classic_junior_total = card_debt[card_debt['Card Type'].isin(['CLASSIC', 'JUNIOR'])]['Total Debt (Loan Amount)'].sum()
others_total = card_debt[~card_debt['Card Type'].isin(['CLASSIC', 'JUNIOR'])]['Total Debt (Loan Amount)'].sum()

print(card_debt)
print(f"\n📊 Debt Comparison:")
print(f"CLASSIC + JUNIOR Debt: ₹{classic_junior_total:,.2f}")
print(f"Other Cards Debt: ₹{others_total:,.2f}")



  Card Type  Total Debt (Loan Amount)
0   CLASSIC                  19791540
2    JUNIOR                   4661376
1      GOLD                   2590092

📊 Debt Comparison:
CLASSIC + JUNIOR Debt: ₹24,452,916.00
Other Cards Debt: ₹2,590,092.00


## How will you analyze the performance of Mid age vs adults in terms of loan repayments

In [57]:
status_map = {0: 'Defaulted', 1: 'Paid with Delay', 2: 'Paid on Time'}
df_Loan['loan_status'] = df_Loan['status'].map(status_map).fillna('Ongoing')
loan_disp = df_Loan.merge(df_Disposition[['account_id', 'client_id']], on='account_id', how='left')
loan_clients = loan_disp.merge(df_Client[['client_id', 'age_levels']], on='client_id', how='left')
loan_clients_filtered = loan_clients[loan_clients['age_levels'].isin(['MID AGE', 'ADULT'])]
performance = loan_clients_filtered.groupby(['age_levels', 'loan_status']).size().reset_index(name='count')
total_by_age = performance.groupby('age_levels')['count'].transform('sum')
performance['percentage'] = (performance['count'] / total_by_age * 100).round(2)
performance


,age_levels,loan_status,count,percentage
0,ADULT,Ongoing,205,100.0


In [104]:
df_Transaction.head()

,account_id,date,type,operation,amount,balance,k_symbol
0,2378,1993-01-01,CREDIT,CREDIT IN CASH,700.0,700.0,SIPO
1,576,1993-01-01,CREDIT,CREDIT IN CASH,900.0,900.0,SIPO
2,704,1993-01-01,CREDIT,CREDIT IN CASH,1000.0,1000.0,SIPO
3,704,1993-01-01,CREDIT,CREDIT IN CASH,1000.0,1000.0,SIPO
4,3818,1993-01-01,CREDIT,CREDIT IN CASH,600.0,600.0,SIPO


In [108]:
df_Card.head()

,card_id,disp_id,type,issued
0,1005,9285,CLASSIC,931107
1,104,588,CLASSIC,940119
2,747,4915,CLASSIC,940205
3,70,439,CLASSIC,940208
4,577,3687,CLASSIC,940215


In [146]:
df_Credit = df_Transaction[df_Transaction['type'].str.upper() == 'CREDIT'].copy()
df_Credit['date'] = pd.to_datetime(df_Credit['date'], errors='coerce')
df_Credit['month'] = df_Credit['date'].dt.to_period('M').astype(str)
df_Credit = df_Credit.merge(df_Account[['account_id', 'district_id']], on='account_id', how='left')
df_Credit = df_Credit.merge(df_District[['Code', 'Region', 'Name']], left_on='district_id', right_on='Code', how='left')
df_Credit_filtered = df_Credit[df_Credit['Region'].str.upper().isin(['SOUTH MORAVIA', 'NORTH MORAVIA', 'PRAGUE'])]
df_Credit_filtered.head()

,account_id,date,type,operation,amount,balance,k_symbol,month,district_id,Code,Region,Name
1,576,1993-01-01,CREDIT,CREDIT IN CASH,900.0,900.0,SIPO,1993-01,55,55,south Moravia,Brno - venkov
2,704,1993-01-01,CREDIT,CREDIT IN CASH,1000.0,1000.0,SIPO,1993-01,55,55,south Moravia,Brno - venkov
3,704,1993-01-01,CREDIT,CREDIT IN CASH,1000.0,1000.0,SIPO,1993-01,55,55,south Moravia,Brno - venkov
4,3818,1993-01-01,CREDIT,CREDIT IN CASH,600.0,600.0,SIPO,1993-01,74,74,north Moravia,Ostrava - mesto
5,3818,1993-01-01,CREDIT,CREDIT IN CASH,600.0,600.0,SIPO,1993-01,74,74,north Moravia,Ostrava - mesto


In [144]:
df_Credit_filtered.to_csv("credit_transactions_moravia_prague.csv")